In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import datetime
from statsmodels.tsa.seasonal import seasonal_decompose

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning:

pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.



In [ ]:
!pip install fbprophet

In [ ]:
from fbprophet import Prophet

In [ ]:
date_quote=pd.read_csv('date_quote_complete.csv', sep=';')

In [ ]:
date_quote['date_month']=date_quote['date'].str[3:5]
date_quote['date_year']=date_quote['date'].str[6:10]
date_quote['date_2']=date_quote['date'].str[6:10]+date_quote['date'].str[3:5]+date_quote['date'].str[0:2]

In [ ]:
date_quote.head()

,date,open,high,low,close,adj close,volume,company,date_month,date_year,date_2
0,02/01/2015,11.613673,11.857728,11.285460,11.319123,8.387552,42539.0,CMIG3,01,2015,20150102
1,05/01/2015,11.319123,11.706246,11.117146,11.117146,8.237885,106705.0,CMIG3,01,2015,20150105
2,06/01/2015,11.150809,11.580010,11.108731,11.108731,8.231650,113359.0,CMIG3,01,2015,20150106
3,07/01/2015,11.243382,11.428527,11.201303,11.234966,8.325192,148294.0,CMIG3,01,2015,20150107
4,08/01/2015,11.361202,11.546347,11.058236,11.058236,8.194233,46579.0,CMIG3,01,2015,20150108


In [ ]:
date_quote['date_month']=date_quote['date_month'].astype(int)
date_quote['date_year']=date_quote['date_year'].astype(int)

In [ ]:
def create_contributions(df):
    df['contributions']=0
    contributions=0
    for year in range(2014,2022):
        for month in range(3, 13, 3):
            contributions=contributions+1
            # Condition
            conditions = [
                (df['date_year'] == year) & (df['date_month'] == month)]
            choices = [contributions]
            df['contributions'] = np.select(conditions, choices, default=df['contributions'])
    return df

In [ ]:
date_quote_2=create_contributions(date_quote) 

In [ ]:
date_quote_2=date_quote_2.sort_values(by=['company', 'date_2'], ascending=True)

In [ ]:
date_quote_contributions=date_quote_2.drop_duplicates(subset=['company', 'contributions'], keep='first')
date_quote_contributions=date_quote_contributions[date_quote_contributions['contributions']!=0]

In [ ]:
# Add contributions
date_quote_contributions_clean=date_quote_contributions[['date_2', 'company', 'contributions']]

In [ ]:
date_quote=date_quote.rename(columns={'contributions': 'contributions_old'})

In [ ]:
date_quote_cont=date_quote.merge(date_quote_contributions_clean, left_on=['date_2', 'company'], 
                                 right_on=['date_2', 'company'], how='left')

In [ ]:
date_quote_cont=date_quote_cont.sort_values(by=['company', 'date_2'], ascending=True)

In [ ]:
# Add final value and difference in days
date_quote_contributions_clean_2=date_quote_contributions[['date', 'company','close','contributions']].reset_index(drop=True)
date_quote_contributions_clean_2['contributions']=date_quote_contributions_clean_2['contributions']-1


date_quote_contributions_clean_2 = date_quote_contributions_clean_2.rename(columns={'date': 'date_fim', 'close': 'close_fim'})

In [ ]:
date_quote_contributions_clean_2

,date_fim,company,close_fim,contributions
0,05/03/2014,CMIG3,11.226550,0
1,02/06/2014,CMIG3,13.927990,1
2,01/09/2014,CMIG3,15.653210,2
3,01/12/2014,CMIG3,11.967130,3
4,02/03/2015,CMIG3,10.772102,4
...,...,...,...,...
2246,01/09/2020,HYPE3,32.160000,26
2247,01/12/2020,HYPE3,31.400000,27
2248,01/03/2021,HYPE3,31.450001,28
2249,01/06/2021,HYPE3,36.820000,29


In [ ]:
date_quote_cont_2=date_quote_cont.merge(date_quote_contributions_clean_2, left_on=['contributions', 'company'],
                                        right_on=['contributions', 'company'], how='left')

In [ ]:
 date_quote_cont_2['date']=pd.to_datetime(date_quote_cont_2['date'], format='%d/%m/%Y')
 date_quote_cont_2['date_fim']=pd.to_datetime(date_quote_cont_2['date_fim'], format='%d/%m/%Y')
 date_quote_cont_2['days'] = (date_quote_cont_2['date_fim'] - date_quote_cont_2['date']).dt.days

In [ ]:
all_contributions=date_quote_contributions.groupby("contributions")["date"].count()
all_contributions

contributions
1     69
2     63
3     64
4     67
5     69
6     69
7     63
8     70
9     72
10    70
11    66
12    70
13    71
14    69
15    74
16    72
17    71
18    71
19    69
20    75
21    73
22    75
23    76
24    79
25    74
26    75
27    81
28    82
29    81
30    89
31    82
Name: date, dtype: int64

In [ ]:
companies_list=date_quote_contributions.groupby("company")["date"].count().reset_index()
companies_list=companies_list['company'].values.tolist()

# Predictions

In [ ]:
def applied_model(df): 
  dataset = df[['date', 'close']].rename(columns = {'date': 'ds', 'close': 'y'})
  modelo = Prophet()
  modelo.fit(dataset)

  futuro = modelo.make_future_dataframe(periods=int(days_ref))
  prediction = modelo.predict(futuro)
  prediction = prediction [['ds', 'yhat']].rename(columns = {'ds': 'date', 'yhat': 'prediction'})

  return prediction

In [ ]:
date_quote_prediction=pd.DataFrame()
for i in range(len(companies_list)):
  date_quote_cont_company = date_quote_cont_2[date_quote_cont_2['company']==companies_list[i]]
  for counter in range(1, len(all_contributions)):
    try:
      date_ref = date_quote_cont_company[date_quote_cont_company['contributions']==counter].iloc[0]['date_2']
      days_ref = date_quote_cont_company[date_quote_cont_company['contributions']==counter].iloc[0]['days']

      date_quote_cont_company_1 = date_quote_cont_company[date_quote_cont_company['date_2']<=date_ref]

      prediction=applied_model(date_quote_cont_company_1)
      prediction=prediction.rename(columns = {'date': 'date_fim'})

      date_quote_prediction_add=date_quote_cont_company_1.merge(prediction, left_on=['date_fim'], right_on=['date_fim'], 
                                                                how='left')
      date_quote_prediction_add = date_quote_prediction_add[~date_quote_prediction_add['prediction'].isna()]
      date_quote_prediction_add=date_quote_prediction_add.sort_values(by=['contributions'])
      date_quote_prediction_add=date_quote_prediction_add.drop_duplicates(subset=['company'], keep='last')

      date_quote_prediction = pd.concat([ date_quote_prediction, date_quote_prediction_add])
    except:
        pass
  print (i)
  date_quote_prediction.to_csv('prediction.csv')

INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seaso

0


INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seaso

1


INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


...

93


INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seaso

94


INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seaso

95


In [ ]:
date_quote_prediction=pd.DataFrame()